# 走行

4つの走行モデルをと環境モデルを用いて走行を切り替えます。

cam0: 走行用カメラ
cam1: 環境認識用カメラ

走行モデルは、大回り直進、大回り左折、小回り直進、小回り左折の4つのモデルを読み込みし走行
環境モデルは、建物を認識

対応デバイス: Jetson Xavier NX, Jetson Orin Nano

## Jetsonの認識

In [1]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

------------------------------------------------------------
Jetson Orin Nanoを認識: I2Cバス番号: 7, Powerモード: MODE_15W(0)に設定します。
------------------------------------------------------------


In [2]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

Docker起動のため電力モードは変更できません。


In [3]:
!echo "jetson" | sudo -S nvpmodel -q

NV Power Mode: 15W
0


In [4]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

Docker起動のためjetson_clocksは起動できません。


## ログの表示用 Widget

In [5]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
from IPython.display import clear_output
import traceback

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value
    
    # UIのクリアと更新
    clear_output(wait=True)

## PWMの値の読み込み

In [6]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

SMBUS='smbus'
BUSNUM=i2c_busnum
SERVO_HZ=60
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0
REVERSE = 0
NORMAL = 1

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

    
if pwm_front >= pwm_back:
    direction = REVERSE
else:
    direction = NORMAL
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

## カメラの読込

この部分でエラーが発生する場合は、Jetsonの再起動をお願いします。<br>
それでも、カメラが認識できない場合は、ケーブルの接続確認をしてください。


In [7]:
CAM0_FPS=60
CAM1_FPS=60

In [8]:
from jetcam.csi_camera import CSICamera

def open_camera():
    global cam0, cam1
    try:
        cam0 = CSICamera(capture_device=0,width=224, height=224, capture_fps=CAM0_FPS)
        cam1 = CSICamera(capture_device=1,width=224, height=224, capture_fps=CAM1_FPS)
    except Exception as e:
        # スタックトレースを含むエラーメッセージを取得
        error_message = f"Error open_camera:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
        write_log(error_message)

## 建物

建物のカテゴリ定義と、1つ前の建物を取得

In [9]:
CATEGORIES = ["mall","diner","bank","school","hotel","supermarket","hospital","campus","etc"]

def get_target(target):
    prebuilding = ""
    if target == "mall":
        prebuilding = "school"
    elif target == "diner":
        prebuilding = "mall"
    elif target == "bank":
        prebuilding = "hospital"
    elif target == "school":
        prebuilding = "google"
    elif target == "hotel":
        prebuilding = "supermarket"
    elif target == "supermarket":
        prebuilding = "mall"
    elif target == "hospital":
        prebuilding = "supermarket"
    elif target == "campus":
        prebuilding = "diner"
    
    return prebuilding

## LED

LEDの色の定義

normal, red, blue, yellow, green, white, orange, purple, lime, pink, off

In [10]:
def init_led():
    global i2c_busnum,i2c
    i2c = smbus.SMBus(i2c_busnum)
    
def change_color(color):
    global i2c
    colors = {
        'normal': 0x10,
        'red':    0x1a,
        'blue':   0x1b,
        'yellow': 0x1c,
        'green':  0x1d,
        'white':  0x1e,
        'orange': 0x1f,
        'purple': 0x20,
        'lime':   0x21,
        'pink':   0x22,
        'off':    0x30,
    }
    if color in colors:
        i2c.read_i2c_block_data(0x08, colors[color])
    else:
        print(f"Error change_color: '{color}' is not a valid color.")

## 走行処理

In [11]:
import threading
import torch
from utils import preprocess
import subprocess
import cv2
import time
from torch2trt import TRTModule
import subprocess
import datetime
import torch.nn.functional as F

record = False
running = False
load = False
speed_ai_flag = False
running_cam0 = False
running_cam1 = False

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = map_rc(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
    
def throttle(speed):
    global pwm_front,pwm_back,THROTTLE_CH,PCA9685, low_gain
    #speed = map_rc(speed, 224, 0, pwm_front, pwm_stop)
    speed = map_rc(speed, 224, 0, pwm_front, pwm_stop + speed_low_gain_slider.value)
    PCA9685.set_channel_value(THROTTLE_CH, speed)

IMG_WIDTH=224

In [12]:
def get_model(status, target):
    if target == "campus" or target == "mall" or target == "diner" or target == "school":
        if status == STATUS_OUT_A:
            name = "model_a"
            return name,model_a_trt
        elif status == STATUS_OUT_B:
            name = "model_b"
            return name,model_b_trt
        elif status == STATUS_IN:
            name = "model_d"
            return name,model_d_trt
        else:
            name = "model_d"
            return name,model_d_trt 
    else:
        if status == STATUS_OUT_A:
            name = "model_a"
            return name,model_a_trt
        elif status == STATUS_OUT_B:
            name = "model_a"
            return name,model_a_trt
        elif status == STATUS_IN:
            name = "model_c"
            return name,model_c_trt
        else:
            name = "model_c"
            return name,model_c_trt 

In [13]:
status = 0

def live_cam0():
    """
    カメラ0(フロントカメラ)からの映像を使用して走行制御を行う関数。
    大回り直進、大回り右折、小回り直進、小回り右折の4つのモデルから選べらたモデルを用いて
    ステアリング操作（handle関数呼び出し）とスロットル制御（throttle関数呼び出し）を行います。
    
    カメラ映像の記録を行う機能も備えています。
    """
    global mode,cam0,target,speed_ai_flag,running_cam0,cam0,IMG_WIDTH,record,selected_model,model_a_trt,preprocess,pwm_stop,save_dir0,save_dir1,num,count_cam0,fps_type,FPS_30, status
    
    try:
        count_cam0 = 1
        num = 1
        frame_count = 0
        # 処理開始時間
        start_time = time.time()
        # 走行用推論実行時間
        process_drive_time = 0
        # 走行用推論実行時間(総計)
        total_process_drive_time = 0
        
        selected_model = model_a_trt

        stop_count = 30
        
        speed = 0
        
        if mode == MODE_RUN:
            status = STATUS_OUT_A
        elif mode == MODE_BACK:
            status = STATUS_BACK
            
    except Exception as e:
        write_log(f"Error live_cam0 init:{e}")

    while running_cam0:
        try:
            # カメラ画像を読込
            img0 = cam0.read()
            if record == True:
                remarked_img0 = img0.copy()
            process_drive_time = time.time()
            img0 = preprocess(img0).half()
            
            
            # 走行用の推論を実行
            output = selected_model(img0).detach().cpu().numpy().flatten()
            x = float(output[0]) * steering_gain_slider.value
            y = float(output[1])
            x = int(IMG_WIDTH * (x / 2.0 + 0.5))
            y = int(IMG_WIDTH * (y / 2.0 + 0.5))
            handle(x)

            if mode == MODE_RUN:
                # スピードを設定(statusによって速度を変える)
                if status == STATUS_OUT_A or status == STATUS_OUT_B:
                    speed = speed_raw_fast_slider.value
                    throttle(speed)
                elif status == STATUS_IN:
                    speed = speed_raw_slow_slider.value
                    throttle(speed)
                elif status == STATUS_TARGET:
                    speed = speed_raw_slow_slider.value
                    throttle(speed)
                elif status == STATUS_ARRIVE:
                    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
                    speed = 0
                    response = send_data(0x0b, 1, "navi_arrived")
                    write_log("Send lora")
                    stop(None)
                    
                model_name, selected_model = get_model(status, target)
            elif mode == MODE_BACK:
                if status == STATUS_BACK:
                    change_color("green")
                    selected_model = model_a_trt
                    speed = speed_raw_fast_slider.value
                    throttle(speed)
                elif status == STATUS_BACK_ARRIVE:
                    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
                    speed = 0
                    change_color("red")
                    response = send_data(0x0b, 1, "back_arrived")
                    write_log("Send lora")
                    stop(None)
            
            # 走行用の推論実行時間を計測
            total_process_drive_time += time.time() - process_drive_time
            
            # 現在の時間を取得
            current_time = time.time()
            
            # 走行のカメラ画像を保存
            if record == True:
                name = "0_0_{:0=5}.jpg".format(count_cam0)
                image_path0 = os.path.join(save_dir0, name)
                cv2.imwrite(image_path0, remarked_img0)

            # カウンターを増加
            count_cam0 += 1
            # フレーム用のカウンターを増加
            frame_count += 1
            

            # 走行時の処理時間計測
            if time.time() - start_time > 3.0:
                fps = frame_count / 3.0
                speed_type = ""
                
                write_log(f"Cam0(走行用) FPS: {fps:.1f}, Speed: {speed:.1f} {speed_type}, Steering Gain: {steering_gain_slider.value},  走行推論: {total_process_drive_time/(fps*3)*1000:.1f}ms ")
                frame_count = 0
                start_time = time.time() 
                total_process_drive_time = 0
            
        except Exception as e:
            # スタックトレースを含むエラーメッセージを取得
            error_message = f"Error live_cam0:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
            write_log(error_message)
            
    if record == True:
        write_log(f"画像を{count_cam0}枚の走行データを保存しました。")

## カメラ1処理

In [14]:
STATUS_WAIT = 99
STATUS_IN = 1
STATUS_OUT_A = 2
STATUS_OUT_B = 3
STATUS_TARGET = 4
STATUS_ARRIVE = 5
STATUS_BACK_WAIT = 6
STATUS_BACK = 7
STATUS_BACK_ARRIVE = 8

MODE_RUN = 1
MODE_BACK = 2
MODE_WAIT = 3

mode = MODE_RUN

In [15]:
last_detect_time = 0  # 最後に検出があった時刻（秒）

def check_non_detection_period(elapsed_time_ms):
    """
    特定の期間内にイベントが検出されていないかどうかをチェックする。
    elapsed_time_msはミリ秒単位で指定する。

    Args:
        elapsed_time_ms (int): 検出がないと判断する期間（ミリ秒）

    Returns:
        bool: 指定された非検出期間を超えていればTrue、そうでなければFalse
    """
    global last_detect_time
    current_time = time.time()  # 現在時刻を取得（秒）
    
    if last_detect_time == 0:  # 初期状態の場合
        last_detect_time = current_time  # 最初の検出時刻を設定

    period_time_ms = (current_time - last_detect_time) * 1000  # 経過時間をミリ秒に変換
    
    if period_time_ms > elapsed_time_ms:
        return True  # 指定された非検出期間を超えている
    else:
        return False  # 指定された非検出期間を超えていない

def update_last_detect_time():
    """
    イベント検出時に最後の検出時刻を現在時刻に更新する。
    """
    global last_detect_time
    last_detect_time = time.time()  # 現在時刻を更新（秒）

In [16]:
def get_stop_time(target):
    if target == CATEGORIES[0]:
        return cat0_slider.value
    elif target == CATEGORIES[1]:
        return cat1_slider.value
    elif target == CATEGORIES[2]:
        return cat2_slider.value
    elif target == CATEGORIES[3]:
        return cat3_slider.value
    elif target == CATEGORIES[4]:
        return cat4_slider.value
    elif target == CATEGORIES[5]:
        return cat5_slider.value
    elif target == CATEGORIES[6]:
        return cat6_slider.value
    elif target == CATEGORIES[7]:
        return cat7_slider.value
    else:
        return 2000

In [17]:
def check_stop_time(current_time, target_detected_time, stop_time_ms):
    # target_detected_time および current_time は秒単位であるため、
    # ミリ秒単位での停止時間を判断するには、秒単位の差をミリ秒単位に変換する
    elapsed_time_ms = (current_time - target_detected_time) * 1000
    if elapsed_time_ms >= stop_time_ms:
        return True
    else:
        return False

In [18]:
def update_status_and_action(current_status, category_index, last_detect, detect_count, prebuilding, target, stop_time, target_detected_time=None):
    """
    現在の状態に基づき、次のアクションを決定し、状態を更新します。ターゲット発見後、指定された秒数でSTOPに遷移します。

    Args:
        current_status (int): 現在の状態
        category_index (int): 現在の認識結果のカテゴリインデックス
        last_detect (int): 前回の認識結果のカテゴリインデックス
        detect_count (int): 連続して認識された回数
        prebuilding (str): ターゲットの1つ前の建物名
        target (str): ターゲットとなる建物名
        stop_time (int): ターゲットを検出してから停止するまでの秒数
        target_detected_time (float, optional): ターゲットを最初に検出した時刻

    Returns:
        tuple: 更新された状態、連続認識回数、ターゲット検出時刻
    """
    global selected_model
    new_status = current_status
    new_detect_count = detect_count
    current_time = time.time()  # 現在時刻を取得

    # STATUS_TARGETに遷移した直後にタイムスタンプを記録
    if current_status == STATUS_TARGET and target_detected_time is None:
        target_detected_time = current_time

    # STATUS_TARGET状態で、指定された秒数が経過した場合にSTATUS_ARRIVEに遷移
    if current_status == STATUS_TARGET:
        if check_stop_time(current_time, target_detected_time, stop_time):
            change_color("red")
            new_status = STATUS_ARRIVE
            PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)    
            target_detected_time = None  # タイムスタンプをリセット
            write_log("STOP!")
            

    if current_status == STATUS_OUT_A:
        if CATEGORIES[category_index] == target:
            if CATEGORIES[last_detect] == target:
                new_detect_count += 1
                if new_detect_count > 10:
                    write_log(f"Detect1 {target}")
                    change_color("yellow")
                    new_status = STATUS_OUT_B
                    new_detect_count = 0
                    update_last_detect_time()

            else:
                new_detect_count = 1
    elif current_status == STATUS_OUT_B:
        if check_non_detection_period(3000) == True:
            if CATEGORIES[category_index] == prebuilding:
                if CATEGORIES[last_detect] == prebuilding:
                    new_detect_count += 1
                    if new_detect_count > 5:
                        write_log(f"Detect2 {prebuilding}")
                        change_color("purple")
                        new_status = STATUS_IN
                        new_detect_count = 0
                else:
                    new_detect_count = 1
    elif current_status == STATUS_IN:
        if CATEGORIES[category_index] == target:
            if CATEGORIES[last_detect] == target:
                new_detect_count += 1
                if new_detect_count > 5:
                    change_color("orange")
                    write_log(f"Detect3 {target}")
                    new_status = STATUS_TARGET
                    new_detect_count = 0
            else:
                new_detect_count = 1
    
    return new_status, new_detect_count, target_detected_time

In [19]:
def update_status_and_action_back(current_status, category_index, last_detect, detect_count, prebuilding, target, stop_time, target_detected_time=None):
    """
    現在の状態に基づき、次のアクションを決定し、状態を更新します。ターゲット発見後、指定された秒数でSTOPに遷移します。

    Args:
        current_status (int): 現在の状態
        category_index (int): 現在の認識結果のカテゴリインデックス
        last_detect (int): 前回の認識結果のカテゴリインデックス
        detect_count (int): 連続して認識された回数
        prebuilding (str): ターゲットの1つ前の建物名
        target (str): ターゲットとなる建物名
        stop_time (int): ターゲットを検出してから停止するまでの秒数
        target_detected_time (float, optional): ターゲットを最初に検出した時刻

    Returns:
        tuple: 更新された状態、連続認識回数、ターゲット検出時刻
    """
    new_status = current_status
    new_detect_count = detect_count
    current_time = time.time()  # 現在時刻を取得

    if current_status == STATUS_BACK:
        if check_non_detection_period(3000) == True:
            if CATEGORIES[category_index] == "hotel":
                write_log(f"Detect1 hotel")
                if CATEGORIES[last_detect] == "hotel":
                    new_detect_count += 1
                    if new_detect_count > 10:
                        write_log(f"Detect2 hotel")
                        change_color("pink")
                        new_status = STATUS_BACK_ARRIVE
                        new_detect_count = 0
                        update_last_detect_time()

                else:
                    new_detect_count = 1
    
    return new_status, new_detect_count, target_detected_time

In [20]:
def live_cam1():
    """
    カメラ1(サイドカメラ)からの映像を使用して環境認識を行う関数。
    クラス分類して認識した結果から現在の状態を変えていく
    """
    global mode,target, cam1, speed_ai_flag,running_cam1,cam1,IMG_WIDTH,record,model_a_trt,model_b_trt,model_c_trt,model_d_trt,model_e_trt,preprocess,pwm_stop,save_dir0,save_dir1,num,count_cam1,fps_type,FPS_30,status
    
    try:
        # 現在の環境情報の認識(live_cam0とも共有する)
        if mode == MODE_RUN:
            status = STATUS_OUT_A
        elif mode == MODE_BACK:
            status = STATUS_BACK
            
        # 認識回数
        detect_count = 0    
        # 環境情報の認識時間計測
        total_process_detect_time = 0
        # カウンター
        count_cam1 = 1
        # フレーム用カウンター
        frame_count = 0
        start_time = time.time()
        last_detect_time = 0
        last_detect = 0
        
        init_led()
        change_color("blue")
        
        # 停止するまでのカウント
        stop_time = get_stop_time(target)
        
        target_detected_time = None
        last_detect_time = 0
    except Exception as e:
        write_log(f"Error live_cam1 init:{e}")

    while running_cam1:
        try:
            # サイドカメラ画像の読込
            img1 = cam1.read()
            if record == True:
                remarked_img1 = img1.copy()
            
            # 環境情報の認識開始時間
            process_detect_time = time.time()
            # 環境情報の推論
            img1 = preprocess(img1).half()
            output = model_e_trt(img1).detach()
            output = F.softmax(output, dim=1).cpu().numpy().flatten()
            category_index = output.argmax()
            # 環境情報の認識時間
            total_process_detect_time += time.time() - process_detect_time 
            
            # 現在の時間を取得
            current_time = time.time()
            
            if mode == MODE_RUN:
                # ターゲットの1つ前の建物名を取得
                prebuilding = get_target(target)
                #last_detect_time = 0
                # 環境認識後の処理（動画作成時も同じロジックを使う)
                status, detect_count, target_detected_time = update_status_and_action(status, category_index, last_detect, detect_count, prebuilding, target, stop_time, target_detected_time)
                last_detect = category_index
            elif mode == MODE_BACK:
                #last_detect_time = 0
                prebuilding = get_target(target)
                status, detect_count, target_detected_time = update_status_and_action_back(status, category_index, last_detect, detect_count, prebuilding, target, stop_time, target_detected_time)
                write_log(f"{status}")
                last_detect = category_index               
            
            
            # 走行のカメラ画像を保存
            if record == True:
                name = "0_0_{:0=5}.jpg".format(count_cam1)
                image_path1 = os.path.join(save_dir1, name)
                cv2.imwrite(image_path1, remarked_img1)
                
            count_cam1 += 1
            frame_count += 1

            # 走行時の処理時間計測
            if time.time() - start_time > 3.0:
                fps = frame_count / 3.0
                speed_type = ""
                if speed_ai_flag == False:
                    speed_type = f"(固定)"
                else:
                    speed_type = f"(推論)"

                write_log(f"Cam1(環境用) FPS: {fps:.1f}, 環境推論: {total_process_detect_time/(fps*3)*1000:.1f}ms")
                frame_count = 0
                start_time = time.time() 
                total_process_detect_time = 0
        except Exception as e:
            error_message = f"Error live_cam1:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
            
    if record == True:
        write_log(f"画像を{cam1_count}枚の走行データを保存しました。")

In [21]:
def start_cameras():
    global cam0, cam1, running_cam0, running_cam1, execute_thread_cam0, execute_thread_cam1
    
    open_camera()
    
    # Cam0を起動
    running_cam0 = True
    execute_thread_cam0 = threading.Thread(target=live_cam0)
    execute_thread_cam0.start()
    write_log("Start cam0")

    # Cam1を起動
    running_cam1 = True
    execute_thread_cam1 = threading.Thread(target=live_cam1)
    execute_thread_cam1.start()
    write_log("Start cam1")

def setup_save_directory(base_path):
    # 指定された基本パスに基づいて保存ディレクトリを作成し、ログに記録します。
    if not os.path.exists(base_path):
        subprocess.call(['mkdir', '-p', base_path])
    write_log(f"{base_path}にデータを保存します。")

def run(change):
    global mode, running, start_time, save_dir0, save_dir1, load_model_a_trt, load_model_b_trt, load_model_c_trt, load_model_d_trt, model_e_model
    
    if not (load_model_a_trt and load_model_b_trt and load_model_c_trt and model_d_trt and model_e_trt):
        write_log("モデルが読み込まれていません")
        return
    if not running:
        write_log("AIが起動しました。")
        if record:
            if name_widget.value != "":
                base_path = "camera/" + name_widget.value
                # Cam0とCam1の保存先を設定
                save_dir0 = f"{base_path}_cam0/xy/"
                save_dir1 = f"{base_path}_cam1/xy/"
                
                setup_save_directory(save_dir0)
                setup_save_directory(save_dir1)
            else:
                write_log("【Error】 映像の保存先を入力してください。")
                return
        write_log("カメラを起動中...")
        start_cameras()
        start_time = time.time()
        
        
def stop(change):
    global running_cam0,running_cam1,execute_thread_cam0,execute_thread_cam1,end_time,start_time,count_cam0,pwm_stop
    if running_cam0 == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count_cam0/int(end_time)
            process_time = int((end_time/count_cam0)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count_cam0) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        running_cam0 = False
        running_cam1 = False
        execute_thread_cam0.join()
        execute_thread_cam1.join()
        write_log("カメラの停止処理")
        stop_camera(None)
        change_color("white")
        
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

## UI

In [22]:
model_a_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_a_time_widget = ipywidgets.Label(description='作成日時')
load_a_button = ipywidgets.Button(description='走行モデルを読込み')
model_b_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_b_time_widget = ipywidgets.Label(description='作成日時')
load_b_button = ipywidgets.Button(description='走行モデルを読込み')
model_c_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_c_time_widget = ipywidgets.Label(description='作成日時')
load_c_button = ipywidgets.Button(description='走行モデルを読込み')
model_d_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_d_time_widget = ipywidgets.Label(description='作成日時')
load_d_button = ipywidgets.Button(description='走行モデルを読込み')
model_e_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_e_time_widget = ipywidgets.Label(description='作成日時')
load_e_button = ipywidgets.Button(description='環境モデルを読込み')
run_lora_button = ipywidgets.Button(description='LoRa受信走行開始')
stop_lora_button = ipywidgets.Button(description='LoRa受信走行停止')
run_button = ipywidgets.Button(description='走行開始')
stop_button = ipywidgets.Button(description='停止')
pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')
check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')
pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')
name_widget = ipywidgets.Text(description='映像の保存名')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_low_gain_slider = ipywidgets.FloatSlider(description='Speed Low gain', min=0, max=20, step=1, value=20, orientation='horizontal')
speed_gain_slider = ipywidgets.FloatSlider(description='Speed gain', min=0.1, max=3.0, step=0.1, value=1.0, orientation='horizontal')
speed_raw_fast_slider = ipywidgets.IntSlider(description='Speed fast', min=1, max=224, step=1, value=224, orientation='horizontal')
speed_raw_slow_slider = ipywidgets.IntSlider(description='Speed slow', min=1, max=224, step=1, value=120, orientation='horizontal')
steering_gain_slider = ipywidgets.FloatSlider(description='Steering gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')
speed_dropbox = ipywidgets.Dropdown(options=["固定値","推論値"], description='Speed')
building_dropbox = ipywidgets.Dropdown(options=CATEGORIES, description='Building')

cat0_slider = ipywidgets.IntSlider(description=CATEGORIES[0], min=1, max=4000, step=100, value=3500, orientation='horizontal')
cat1_slider = ipywidgets.IntSlider(description=CATEGORIES[1], min=1, max=4000, step=100, value=1000, orientation='horizontal')
cat2_slider = ipywidgets.IntSlider(description=CATEGORIES[2], min=1, max=4000, step=100, value=1000, orientation='horizontal')
cat3_slider = ipywidgets.IntSlider(description=CATEGORIES[3], min=1, max=4000, step=100, value=1000, orientation='horizontal')
cat4_slider = ipywidgets.IntSlider(description=CATEGORIES[4], min=1, max=4000, step=100, value=3000, orientation='horizontal')
cat5_slider = ipywidgets.IntSlider(description=CATEGORIES[5], min=1, max=4000, step=100, value=4000, orientation='horizontal')
cat6_slider = ipywidgets.IntSlider(description=CATEGORIES[6], min=1, max=4000, step=100, value=1000, orientation='horizontal')
cat7_slider = ipywidgets.IntSlider(description=CATEGORIES[7], min=1, max=4000, step=100, value=1000, orientation='horizontal')

In [23]:
target = CATEGORIES[0]

def set_target(change):
    global target
    target = change.new  
    write_log(f"target: {target}")

building_dropbox.observe(set_target, names='value')

In [24]:
import numpy as np

def load_model(widget, model_var_name):
    try:
        write_log(f"{widget.value}の読込を実行します(初回は時間がかかります)。")
        model = TRTModule()
        model.load_state_dict(torch.load(widget.value))
        model(preprocess(np.zeros((224, 224, 3)).astype(np.uint8)))
        write_log(f"{widget.value}の読込に成功しました。")
        globals()[model_var_name] = model  # 成功した場合、グローバル変数にモデルをセット
        load_flag_var_name = f"load_{model_var_name}"
        write_log(f"{load_flag_var_name}の読込に成功しました。")
        globals()[load_flag_var_name] = True  # 対応するloadフラグをTrueにセット
        get_jetson_nano_memory_usage()
    except Exception as e:
        write_log(f"【Error】 {e} : {widget.value} の読込に失敗しました。")
        
# モデル読み込み関数を各ボタンのクリックイベントにバインドする例
load_a_button.on_click(lambda change: load_model(model_a_widget, 'model_a_trt'))
load_b_button.on_click(lambda change: load_model(model_b_widget, 'model_b_trt'))
load_c_button.on_click(lambda change: load_model(model_c_widget, 'model_c_trt'))
load_d_button.on_click(lambda change: load_model(model_d_widget, 'model_d_trt'))
load_e_button.on_click(lambda change: load_model(model_e_widget, 'model_e_trt'))

In [25]:
def model_list(type):
    try:
        files = glob.glob(type + '/*.pth', recursive=True)
        
        # Initialize dictionaries to hold the first matching file for each widget
        first_match = {
            "model_a_widget": None,
            "model_b_widget": None,
            "model_c_widget": None,
            "model_d_widget": None
        }
        
        for file in files:
            if "out_straight" in file and first_match["model_a_widget"] is None:
                first_match["model_a_widget"] = file
            elif "out_right" in file and first_match["model_b_widget"] is None:
                first_match["model_b_widget"] = file
            elif "in_straight" in file and first_match["model_c_widget"] is None:
                first_match["model_c_widget"] = file
            elif "in_right" in file and first_match["model_d_widget"] is None:
                first_match["model_d_widget"] = file

        ts = os.path.getctime(files[0]) if files else 0
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        
        if type == "model_trt":
            model_a_widget.options = files
            model_a_widget.value = first_match["model_a_widget"]
            model_a_time_widget.value = s if first_match["model_a_widget"] else ""
            
            model_b_widget.options = files
            model_b_widget.value = first_match["model_b_widget"]
            model_b_time_widget.value = s if first_match["model_b_widget"] else ""
            
            model_c_widget.options = files
            model_c_widget.value = first_match["model_c_widget"]
            model_c_time_widget.value = s if first_match["model_c_widget"] else ""
            
            model_d_widget.options = files
            model_d_widget.value = first_match["model_d_widget"]
            model_d_time_widget.value = s if first_match["model_d_widget"] else ""
        
        elif type == "model_c_trt":
            model_e_widget.options = files
            # If there's a specific file you want to set as active for model_e_widget, add the logic here
            model_e_time_widget.value = s
        
    except Exception as e:
        if type == "model_trt":
            model_a_widget.options = []
            model_b_widget.options = []
            model_c_widget.options = []
            model_d_widget.options = []
        elif type == "model_c_trt":
            model_e_widget.options = []
        write_log(f"Error model_list:{e}")
        
model_list("model_trt")
model_list("model_c_trt")

def update_model_time_widget(widget, time_widget):
    """指定されたウィジェットのファイル選択が変更された際の処理。ファイルの作成時間を表示ウィジェットに設定する。"""
    file = widget.value
    try:
        ts = os.path.getctime(file)
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        time_widget.value = s
    except Exception as e:
        time_widget.value = "Error update_model_time_widget: " + str(e)

# 各モデル選択ウィジェットの変更を監視し、対応する時刻表示ウィジェットを更新
model_a_widget.observe(lambda change: update_model_time_widget(model_a_widget, model_a_time_widget), names='value')
model_b_widget.observe(lambda change: update_model_time_widget(model_b_widget, model_b_time_widget), names='value')
model_c_widget.observe(lambda change: update_model_time_widget(model_c_widget, model_c_time_widget), names='value')
model_d_widget.observe(lambda change: update_model_time_widget(model_d_widget, model_d_time_widget), names='value')
model_e_widget.observe(lambda change: update_model_time_widget(model_e_widget, model_e_time_widget), names='value')

def on_video(change):
    global record
    record^=True

def on_fixed_value_change(change):
    global speed_ai_flag, speed_gain_slider, speed_raw_fast_slider,speed_raw_slow_slider
    if change['new'] == "固定値":
        speed_gain_slider.layout.visibility = 'hidden'
        speed_raw_fast_slider.layout.visibility = 'visible'
        speed_raw_slow_slider.layout.visibility = 'visible'
        speed_gain_slider.disabled = True
        speed_raw_fast_slider.disabled = False
        speed_raw_slow_slider.disabled = False
        speed_ai_flag = False
    elif change['new'] == "推論値":
        speed_gain_slider.layout.visibility = 'visible'
        speed_raw_fast_slider.layout.visibility = 'hidden'
        speed_raw_slow_slider.layout.visibility = 'hidden'
        speed_gain_slider.disabled = False
        speed_raw_fast_slider.disabled = True
        speed_raw_slow_slider.disabled = True
        speed_ai_flag = True

speed_gain_slider.layout.visibility = 'hidden'
speed_raw_fast_slider.layout.visibility = 'visible'
speed_raw_slow_slider.layout.visibility = 'visible'

speed_gain_slider.disabled = True
speed_raw_fast_slider.disabled = False
speed_raw_slow_slider.disabled = False

speed_dropbox.observe(on_fixed_value_change, names='value')

run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)

In [26]:
import subprocess
import re

used_memory_widget = ipywidgets.IntText(description='Useメモリ', value=1)
total_memory_widget = ipywidgets.IntText(description='全メモリ', value=1)
memory_button = ipywidgets.Button(description='使用メモリ量の取得')

def get_jetson_nano_memory_usage(event=None):
    command = 'tegrastats'
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        
        mem_usage_pattern = re.compile(r'RAM (\d+)/(\d+)MB')
        
        max_lines_to_read = 10
        for _ in range(max_lines_to_read):
            line = process.stdout.readline()
            if not line:
                break 
            matches = mem_usage_pattern.search(line)
            if matches:
                used_memory_widget.value = int(matches.group(1))
                total_memory_widget.value = int(matches.group(2))
                process.kill()
                return
        
        process.kill()  
        return

    except subprocess.CalledProcessError as e:
        return

get_jetson_nano_memory_usage()
memory_button.on_click(get_jetson_nano_memory_usage)

In [27]:
import time

release_button = ipywidgets.Button(description='Camera開放')

def stop_camera(c):
    global cam0,cam1
    cam0.running = False
    cam1.running = False
    time.sleep(0.1)
    cam1.cap.release()
    cam0.cap.release()
    write_log("カメラ0,1を開放しました。")

release_button.on_click(stop_camera)

## LoRa関連

In [28]:
import serial
import time

import threading
import re  # 正規表現モジュール

def run_lora_threaded(c):
    global thread_lora
    write_log("LoRaスレッドを起動")
    try:
        thread_lora= threading.Thread(target=run_lora, args=(c,))
        thread_lora.start()
    except Exception as e:
        # スタックトレースを含むエラーメッセージを取得
        error_message = f"Error run_lora_thread:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
        write_log(error_message)

def stop_lora(c):
    global listening, thread_lora
    write_log("Stop lora")
    listening = False
    thread_lora.join()
    stop(None)
    
def run_lora(c=None):
    global listening, ser, target, mode
    listening = True  # リスニングを有効にする
    init_led()
    change_color("purple")
    port = '/dev/ttyUSB0'  # シリアルポートのデバイス名
    baudrate = 9600        # ボーレート（デバイスに合わせて設定）
    timeout = 1            # タイムアウトの秒数
    write_log("LoRaのリスニングを開始")

    try:
        # シリアル接続を開始する前にRTSを制御してLoRaモジュールをリセット
        pre_reset_ser = serial.Serial(port, baudrate, timeout=timeout)
        pre_reset_ser.setRTS(False)  # RTSをLOWに設定
        time.sleep(0.1)  # 少し待機
        pre_reset_ser.setRTS(True)  # RTSをHIGHに設定
        pre_reset_ser.close()  # RTS制御用の接続を閉じる

        # シリアルポートを開く
        with serial.Serial(port, baudrate, timeout=timeout) as ser:
            write_log(f"{port} でリスニング中...")
            try:
                while listening:
                    if ser.in_waiting > 0:
                        data = ser.readline().decode('utf-8').strip()  # データを文字列として読み取り
                        write_log(f"受信データ: {data}")

                        # 正規表現を使って送信者IDと目的の値を抽出
                        match = re.search(r'\[([0-9]+)\]\s([0-9]+)', data)
                        if match:
                            sender_id = int(match.group(1))  # 送信者ID
                            value = int(match.group(2))  # 目的の値を整数に変換
                            write_log(f"送信者ID: {sender_id}, 抽出した値: {value}")

                            # 抽出した値が0から10の範囲内か判定
                            if 1 <= value <= len(CATEGORIES):
                                write_log(f"値は1〜{len(CATEGORIES)}の範囲内です。")
                                target = CATEGORIES[value-1]  # 取得したデータに基づいて行動
                                mode = MODE_RUN
                                run(None)  # 実際に何かしらの処理を行う関数をここで呼び出す
                            elif value == 9:
                                mode = MODE_BACK
                                write_log("BACK modeで起動")
                                run(None)
                            else:
                                write_log("値は0-10の範囲外です。")

                    time.sleep(0.1)  # CPU使用率の低減
            except Exception as e:
                error_message = f"Error run_lora:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
                write_log(error_message)
            finally:
                write_log("リスニング停止")

    except Exception as e:
        error_message = f"Error run_lora:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
        write_log(error_message)

            
            
run_lora_button.on_click(run_lora_threaded)
stop_lora_button.on_click(stop_lora)

In [29]:
def send_data(send_address, send_ch, data):
    global ser
    data_size = len(data)
    message = bytearray(data_size + 4)  # plus 4 for address and ch, plus 1 for delimiter
    message[0] = (send_address >> 8) & 0xFF  # high byte of send_address
    message[1] = send_address & 0xFF  # low byte of send_address
    message[2] = send_ch

    message[3:data_size+3] = data.encode('utf-8')  # assuming data is a str object

    # add delimiter
    message[data_size+3] = ord('\n')

    # Write message
    ser.write(message)

    # Wait for data to become available
    timeout = time.time() + 1  # 1 second timeout
    while ser.in_waiting <= 0 and time.time() < timeout:
        time.sleep(0.01)  # Wait a bit for data to arrive

    # Read response (adjust the size of the read as needed)
    ser.flush()
    if ser.in_waiting > 0:
        response = ser.read(1)
        return response[0]
    else:
        print("No response received.")
        return None

走行までの流れは以下の通りです。

1. <b>走行モデルを指定してLoadする</b><br>
2. <b>環境モデルを指定してLoadする</b><br>
3. <b>Speedに固定値, Speedに推論値のいずれかを選択する</b><br>
固定値を選んだ場合は固定の速度で走り続けます。推論値を選んだ推論結果を速度に反映します。速度用のアノテーションは13_annotation.ipynbで追加可能です。<br>
4. <b>[オプション] 走行動画を録画する場合は、録画にチェックマークをいれて、保存ファイル名を指定する</b><br>
./runフォルダに保存<br>
5. <b>走行開始ボタンを押して、プロポの裏側のボタンを押して AIモードで自動走行開始する</b><br>
6. <b>終了時は、停止ボタンを押す</b><br>
録画のチェックマークがついている場合は、停止で録画も終了<br>
Speed Inferenceのチェックマークがついている場合は、スロットル量の推論も有効になる<br>
<br>
カメラが60fpsで動いている場合は16ms以内、カメラが30fpsで動いている場合は、33ms以内での処理完了が正常な挙動となります。

In [30]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.使用する走行モデルA(外回り直進)】</b> TensoRTに変換済みのモデルをLoadします。')
title2 = ipywidgets.HTML('<b>【2.使用する走行モデルB(外回り右折)】</b> TensoRTに変換済みのモデルをLoadします。')
title3 = ipywidgets.HTML('<b>【3.使用する走行モデルC(内回り直進)】</b> TensoRTに変換済みのモデルをLoadします。')
title4 = ipywidgets.HTML('<b>【4.使用する走行モデルD(内回り右折)】</b> TensoRTに変換済みのモデルをLoadします。')
title5 = ipywidgets.HTML('<b>【5.使用する環境モデルE】</b> TensoRTに変換済みのモデルをLoadします。')
title6 = ipywidgets.HTML('<b>【6.目標とするとBuilding】</b> 目標のビルを設定します')
title7 = ipywidgets.HTML('<b>【7.停止時間の設定】</b> ターゲットの停止時間を設定')
title8 = ipywidgets.HTML('<b>【8.Steeringゲイン】</b> Steeringのゲイン調整します。周りが悪い時は値を1.0以上にします。')
title9 = ipywidgets.HTML('<b>【9.速度】</b> 速度は固定値か、推論から反映かが選べます。')
title10 = ipywidgets.HTML('<b>【10.録画】</b> 走行中の映像を録画したい場合はチェックマークを選択し、保存データセット名を指定してください。')
title11 = ipywidgets.HTML('<b>【11.走行の開始】</b> 走行開始を押す前にタイヤが空転していのを確認してください。プロポの裏のボタンを変換しAIモードにすると動き始めます。')

data_collection_widget = ipywidgets.VBox([
    separator,
    title5,
    ipywidgets.HBox([model_e_widget,model_e_time_widget,load_e_button]),
    process_widget,
    separator,
    title1,
    ipywidgets.HBox([model_a_widget,model_a_time_widget,load_a_button]),
    process_widget,
    separator,
    title2,
    ipywidgets.HBox([model_b_widget,model_b_time_widget,load_b_button]),
    process_widget,
    separator,
    title3,
    ipywidgets.HBox([model_c_widget,model_c_time_widget,load_c_button]),
    process_widget,
    separator,
    title4,
    ipywidgets.HBox([model_d_widget,model_d_time_widget,load_d_button]),
    process_widget,
    separator, 
    title6,
    building_dropbox,
    separator,
    title7,
    ipywidgets.HBox([cat0_slider,cat1_slider,cat2_slider]),
    ipywidgets.HBox([cat3_slider,cat4_slider,cat5_slider]),
    ipywidgets.HBox([cat6_slider,cat7_slider]),
    separator,
    title8,
    ipywidgets.HBox([steering_gain_slider]),
    separator,
    title9,
    ipywidgets.HBox([speed_dropbox]),
    ipywidgets.HBox([speed_raw_fast_slider,speed_gain_slider]),
    ipywidgets.HBox([speed_raw_slow_slider,speed_gain_slider]),
    ipywidgets.HBox([speed_low_gain_slider]),
    separator,
    title10,
    ipywidgets.HBox([record_box,name_widget]),
    separator,
    title11,
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    ipywidgets.HBox([run_lora_button,stop_lora_button]),
    ipywidgets.HBox([run_button, stop_button]),
    process_widget,
    separator,
])
display(data_collection_widget)